<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/faiss_hnsw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install libomp-dev
!pip install --upgrade faiss-cpu faiss-gpu


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (595 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a14

1. https://www.pinecone.io/learn/series/faiss/hnsw/
2. https://www.youtube.com/watch?v=QvKMwLjdK-s




# 分层可导航小世界 (HNSW)

![分层可导航小世界](https://cdn.sanity.io/images/vr8gru94/production/d6e3a660654d9cb55f7ac137a736539e227296b6-1920x1080.png)

[分层可导航小世界 (HNSW) 图是矢量相似性搜索](https://www.pinecone.io/learn/what-is-similarity-search/)中表现最好的索引之一[1]。HNSW 是一项非常受欢迎的技术，它一次又一次地产生最先进的性能，具有超快的搜索速度和出色的召回率。

然而，尽管近似最近邻 (ANN) 搜索是一种流行且强大的算法，但理解它的工作原理却远非易事。


本文帮助揭开 HNSW 的神秘面纱，并以易于理解的方式解释了这种智能算法。在本文的最后，我们将了解如何使用Faiss实现 HNSW以及哪些参数设置可以为我们提供所需的性能。

## HNSW 的基础

我们可以将[ANN 算法](https://www.pinecone.io/learn/what-is-similarity-search/)分为三个不同的类别；树、散列和图。HNSW 属于*图表*类别。更具体地说，它是一个*邻近图*，其中两个顶点根据它们的邻近度进行链接（较近的顶点被链接）——通常以欧几里德距离定义。

*从“邻近”*图到*“分层可导航小世界”*图，复杂性发生了显着的飞跃。我们将描述对 HNSW 贡献最大的两种基本技术：概率跳跃列表和可导航的小世界图。

### 概率跳过列表

概率跳跃列表*早*在 1990 年就由*William Pugh*提出[2]。它允许像排序数组一样进行快速搜索，同时使用链表结构轻松（快速）插入新元素（这是排序数组不可能实现的）。

跳过列表通过构建多层链接列表来工作。在第一层，我们发现*跳过*许多中间节点/顶点的链接。当我们向下移动层时，每个链接的*“跳过”*数量会减少。

*为了搜索跳跃列表，我们从具有最长“跳跃”*的最高层开始，并沿着边缘向右移动（如下）。如果我们发现当前节点“key”大于*我们*正在搜索的键——我们就知道我们已经超出了我们的目标，所以我们向下移动到下一个级别的前一个*节点*。

![概率跳表结构，我们从顶层开始。 如果我们当前的键大于我们正在搜索的键（或者到达末尾），我们就会进入下一层。](https://cdn.sanity.io/images/vr8gru94/production/9065d31e1b2e33ca697a56082f0ece7eff1c2d9b-1920x500.png)

概率跳表结构，我们从顶层开始。如果我们当前的键大于我们正在搜索的键（或者到达末尾），我们就会进入下一层。

HNSW 继承了相同的分层格式，最高层具有较长的边缘（用于快速搜索），较低层具有较短的边缘（用于精确搜索）。

### 可导航的小世界图

使用*可导航小世界*(NSW) 图的矢量搜索是在 2011-14 年的几篇论文中介绍的[4,5,6]。这个想法是，如果我们采用邻近图但构建它以便我们同时具有远程和短程链接，那么搜索时间就会减少到（多/）对数复杂度。

图中的每个顶点都连接到其他几个顶点。我们将这些连接的顶点称为*朋友*，每个顶点都保存一个*朋友列表*，创建我们的图。

当搜索 NSW 图时，我们从预定义的*入口点*开始。该入口点连接到几个附近的顶点。我们确定这些顶点中的哪个最接近我们的查询向量并移动到那里。

![通过 NSW 图的搜索过程。 从预定义的入口点开始，算法贪婪地遍历到更接近查询向量的连接顶点。](https://cdn.sanity.io/images/vr8gru94/production/5ca4fca27b2a9bf89b06748b39b7b6238fd4548c-1920x1080.png)

通过 NSW 图的搜索过程。从预定义的入口点开始，算法贪婪地遍历到更接近查询向量的连接顶点。

我们通过识别每个朋友列表中最近的相邻顶点来重复从一个顶点移动到另一个顶点的*贪婪路由搜索过程。*最终，我们将找不到比当前顶点更近的顶点——这是局部最小值，并且充当我们的停止条件。

------

*可导航的小世界模型被定义为使用贪婪路由的具有（聚/）对数复杂度的任何网络。当图不可导航时，对于较大的网络（1-10K+ 顶点），贪婪路由的效率会下降* *[7]。*

------

路由（实际上是我们通过图表*所*采取的路线）由两个阶段组成。我们从“缩小”阶段开始，在该阶段中，我们会经过低度数的顶点（度数是顶点所具有的链接数量），然后是“放大”阶段，在该阶段中，我们会经过较高度数的顶点 [8] 。

![高度顶点有很多链接，而低度顶点有很少链接。](https://cdn.sanity.io/images/vr8gru94/production/d65da887accc1d8d577f236459b16946c9f71a96-1920x960.png)

高度顶点有很多链接，而低度顶点有很少链接。

我们的*停止条件*是在当前顶点的好友列表中找不到更近的顶点。因此，在*缩小*阶段时，我们更有可能达到局部最小值并过早停止（链接较少，找到较近顶点的可能性较小）。

为了最大限度地减少提前停止的概率（并增加召回率），我们可以增加顶点的平均度，但这会增加网络复杂性（和搜索时间）。所以我们需要平衡召回率和搜索速度之间的平均顶点度。

另一种方法是在高度顶点上开始搜索（首先*放大*）。对于新南威尔士州来说，这*确实*提高了低维数据的性能。我们将会看到，这也是 HNSW 结构中的一个重要因素。

### 创建HNSW

HNSW 是 NSW 的自然演变，它借鉴了 Pugh 概率跳表结构的分层多层的灵感。

将层次结构添加到 NSW 会生成一个图表，其中链接在不同层之间分开。在顶层，我们有最长的链接，在底层，我们有最短的链接。

![HNSW 的分层图，顶层是我们的入口点，仅包含最长的链接，当我们向下移动时，链接长度变得更短且数量更多。](https://cdn.sanity.io/images/vr8gru94/production/42d4a3ffc43e5dc2758ba8e5d2ef29d4c4d78254-1920x1040.png)

HNSW 的分层图，顶层是我们的入口点，仅包含最长的链接，当我们向下移动时，链接长度变得更短且数量更多。

在搜索过程中，我们进入顶层，在那里我们找到最长的链接。这些顶点往往是度数较高的顶点（链接跨多个层分开），这意味着我们默认从NSW 描述的*放大*阶段开始。

我们遍历每一层中的边，就像我们在新南威尔士州所做的那样，贪婪地移动到最近的顶点，直到找到局部最小值。与 NSW 不同，此时，我们转移到较低层中的当前顶点并再次开始搜索。我们重复这个过程，直到找到底层 -*第 0 层*的局部最小值。

![通过 HNSW 图的多层结构的搜索过程。](https://cdn.sanity.io/images/vr8gru94/production/e63ca5c638bc3cd61cc1cd2ab33b101d82170426-1920x1080.png)

通过 HNSW 图的多层结构的搜索过程。

## 图构建

在图构建过程中，向量被一一迭代地插入。层数由参数*L*表示。*在给定层插入向量的概率由“层乘数” m_L*归一化的概率函数给出，其中*m_L = ~0*表示向量仅在*第 0 层*插入。

![对每一层（第 0 层除外）重复概率函数。 该向量被添加到其插入层及其下面的每一层。](https://cdn.sanity.io/images/vr8gru94/production/f105cb148aae44f77fa7e3df7b7f8c0256bcbec4-1920x980.png)

对每一层（第 0 层除外）重复概率函数。该向量被添加到其插入层及其下面的每一层。

HNSW 的创建者发现，当我们最小化跨层共享邻居的重叠时，可以获得最佳性能。*减少 m_L*可以帮助最小化重叠（将更多向量推送到*第 0 层*），但这会增加搜索期间的平均遍历次数。因此，我们使用*m_L*值来平衡两者。*该最佳值的经验法则是* ***1/ln(M)*** *[1]*。

图的构建从顶层开始。进入图形后，算法贪婪地遍历边缘，找到与我们插入的向量*q*最近的*ef* - 此时*ef = 1*。

找到局部最小值后，它会向下移动到下一层（就像搜索期间所做的那样）。重复这个过程直到到达我们选择的*插入层*。这里开始第二阶段的建设。

ef值增加到**efConstruction**（我们设置的参数），这意味着将返回更多的最近邻居*。*在第二阶段，这些最近邻居是新插入元素*q的链接的候选者**，并*作为下一层的入口点。

*M 个*邻居被添加为这些候选向量的链接 - 最直接的选择标准是选择最接近的向量。

经过多次迭代后，添加链接时还要考虑两个参数。*M_max*定义一个顶点可以拥有的最大链接数，*M_max0*定义相同但针对*第 0 层*中的顶点。

![分配给每个顶点的链接数量以及 M、M_max 和 M_max0 的效果的说明。](https://cdn.sanity.io/images/vr8gru94/production/dc5cb11ea197ceb4e1f18214066c8c51526b9af5-1920x1080.png)

分配给每个顶点的链接数量以及 M、M_max 和 M_max0 的效果的说明。

*插入的停止条件是在第 0 层*达到局部最小值。




首先，我们需要数据。我们将使用[Sift1M 数据集](http://corpus-texmex.irisa.fr/)，我们可以使用以下命令将其下载并加载到笔记本中：

In [6]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [7]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [8]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [9]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [12]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

In [10]:
xq.shape

(1, 128)

In [11]:
xb.shape

(1000000, 128)

## 实施 HNSW

我们将使用 Facebook AI 相似性搜索 (Faiss) 库实现 HNSW，并测试不同的构建和搜索参数，看看它们如何影响索引性能。

为了初始化 HNSW 索引，我们编写：

In [2]:
import faiss
# setup our HNSW parameters
d = 128  # vector size
M = 32

index = faiss.IndexHNSWFlat(d, M)
print(index.hnsw)

<faiss.swigfaiss_avx2.HNSW; proxy of <Swig Object of type 'faiss::HNSW *' at 0x791f0d774840> >


这样，我们就设置了M参数——插入时添加到每个顶点的邻居数量，但我们缺少M_max和M_max0。

在 Faiss 中，这两个参数在set_default_probas方法中自动设置，在索引初始化时调用。M_max值设置为M，M_max0设置为M*2（在笔记本中查找更多详细信息）。

在使用index.add(xb)构建索引之前，我们会发现层数（或Faiss 中的级别）未设置：



In [3]:
# the HNSW index starts with no levels
index.hnsw.max_level

-1

In [5]:
import numpy as np
# and levels (or layers) are empty too
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

如果我们继续构建索引，我们会发现这两个参数现在都已设置。



In [13]:
index.add(xb)

In [14]:
# after adding our data we will find that the level
# has been set automatically
index.hnsw.max_level

4

In [15]:
# and levels (or layers) are now populated
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

这里我们有图表中的级别数，如max_level所描述的0 -> 4。我们有级别，它显示从0到4的每个级别上的顶点分布（忽略第一个0值）。我们甚至可以找到哪个向量是我们的入口点：



In [16]:
index.hnsw.entry_point

118295

这是 Faiss 风格的 HNSW 图的高级视图，但在测试该索引之前，让我们更深入地了解 Faiss 如何构建此结构。

#### 图结构

当我们初始化索引时，我们传递向量维度**d**和每个顶点的邻居数量**M**。这将调用方法“**set_default_probas**”，并在*levelMult*的位置传递**M**和**1 / log(M)**（相当于上面的*m_L*）。此方法的 Python 等效项如下所示：


In [17]:
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbors count = 0
    cum_nneighbor_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbors is == M on every level except level 0 where == M*2
        nn += M*2 if level == 0 else M
        cum_nneighbor_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbor_per_level

在这里，我们构建两个向量 - allocate_probas 给定层的插入概率 和cum_nneighbor_per_level 分配给不同插入级别的顶点的最近邻居的累积总数。



In [18]:
assign_probas, cum_nneighbor_per_level = set_default_probas(
    32, 1/np.log(32)
)
assign_probas, cum_nneighbor_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])

由此，我们可以看到在级别 0插入向量的概率明显高于在更高级别插入向量的概率（尽管正如我们将在下面解释的那样，该概率并不完全按照此处的定义）。这个函数意味着更高的级别更稀疏，减少“卡住”的可能性，并确保我们从更长范围的遍历开始。

我们的allocate_probas向量由另一种称为random_level 的方法使用- 正是在这个函数中，每个顶点被分配了一个插入级别。



In [19]:
def random_level(assign_probas: list, rng):
    # get random float from 'r'andom 'n'umber 'g'enerator
    f = rng.uniform()
    for level in range(len(assign_probas)):
        # if the random float is less than level probability...
        if f < assign_probas[level]:
            # ... we assert at this level
            return level
        # otherwise subtract level probability and try again
        f -= assign_probas[level]
    # below happens with very low probability
    return len(assign_probas) - 1

我们使用 Numpy 的随机数生成器rng（在下面初始化）在f中生成随机浮点数。对于每个级别，我们检查f是否小于allocate_probas中为该级别指定的概率- 如果是，则这就是我们的插入层。

如果f太高，我们从f中减去allocate_probas值，然后重试下一个级别。这个逻辑的结果是向量最有可能被插入到级别 0。尽管如此，如果没有，在轻松增量水平上插入的可能性就会降低。

最后，如果没有级别满足概率条件，我们将向量插入到最高级别并返回 len(assign_probas) - 1。如果我们比较 Python 实现和 Faiss 实现之间的分布，我们会看到非常相似的结果：



In [20]:
chosen_levels = []
rng = np.random.default_rng(12345)
for _ in range(1_000_000):
    chosen_levels.append(random_level(assign_probas, rng))

In [21]:
np.bincount(chosen_levels)

array([968821,  30170,    985,     23,      1])



![Faiss 实现（左）和 Python 实现（右）中的跨层顶点分布。](https://cdn.sanity.io/images/vr8gru94/production/75658a08c25dabc1405f769c76fd2929c051853b-1920x930.png)

Faiss 实现（左）和 Python 实现（右）中的跨层顶点分布。

Faiss 实现还确保我们在最高层中*始终*至少有一个顶点作为图的入口点。

### HNSW实现

现在我们已经探索了 HNSW 背后的理论以及如何在 Faiss 中实现这一点 - 让我们看看不同参数对我们的召回、搜索和构建时间以及每个参数的内存使用的影响。

我们将修改三个参数：**M**、**efSearch**和**efConstruction**。我们将为 Sift1M 数据集建立索引，您可以使用[此脚本](https://gist.github.com/jamescalam/a09a16c17b677f2cf9c019114711f3bf)下载并准备该数据集。

正如我们之前所做的那样，我们像这样初始化索引：



In [23]:
index = faiss.IndexHNSWFlat(d, M)

其他两个参数efConstruction和efSearch可以在初始化索引后进行修改。



In [24]:
# set HNSW index parameters
efSearch = 32  # depth of layers explored during search
efConstruction = 64  # depth of layers explored during index construction

index.hnsw.efConstruction = efConstruction
index.add(xb)  # build the index
index.hnsw.efSearch = efSearch
# and now we can search
D, I = index.search(xq[:1000], k=1)
print(I,D)

[[932085]] [[54229.]]


我们的**efConstruction**值必须在通过index.add(xb)构建索引之前设置，但efSearch可以在搜索之前随时设置。

我们先来看看召回表现。

![Recall@1 各种 M、efConstruction 和 efSearch 参数的性能。](https://cdn.sanity.io/images/vr8gru94/production/e8c281c3626226a76389fa344a71eb57f70cf879-1920x980.png)

Recall@1 各种 M、efConstruction 和 efSearch 参数的性能。

高M和efSearch值可以对召回性能产生很大影响 - 而且很明显需要合理的efConstruction值。我们还可以增加efConstruction，以在较低的M和efSearch值下实现更高的召回率。

然而，这种表演并不是免费的。与往常一样，我们在召回率和搜索时间之间进行了平衡——让我们来看看。

![搜索 1000 个查询时，各种 M、efConstruction 和 efSearch 参数的搜索时间（以 µs 为单位）。 请注意，y 轴使用对数刻度。](https://cdn.sanity.io/images/vr8gru94/production/876bf66aba408959042888efe72c55db4d6b3b41-1920x980.png)

搜索 1000 个查询时，各种 M、efConstruction 和 efSearch 参数的搜索时间（以 µs 为单位）。请注意，y 轴使用对数刻度。

尽管较高的参数值可以为我们提供更好的召回率，但对搜索时间的影响可能会很大。在这里，我们搜索**1000 个**相似向量 ( **xq[:1000]** )，我们的召回/搜索时间可以从 80%-1ms 到 100%-50ms 不等。如果我们对相当糟糕的召回感到满意，搜索时间甚至可以达到 0.1 毫秒。

如果您一直在关注我们[关于向量相似性搜索的文章](https://www.pinecone.io/learn/)，您可能还记得**efConstruction**[对搜索时间的影响可以忽略不计](https://www.pinecone.io/learn/series/faiss/vector-indexes/)-但这里的情况并非如此......

当我们使用几个查询进行搜索时，efConstruction对搜索时间的影响确实很小。但由于此处使用了1000 个查询， efConstruction的微小影响变得更加显着。

如果您认为您的查询大多是低容量的，那么efConstruction是一个很好的增加参数。*它可以提高召回率，而对搜索时间*影响很小，特别是在使用较低的**M**值时。

![ef 仅搜索一个查询时的构造和搜索时间。 当使用较低的 M 值时，对于不同的 efConstruction 值，搜索时间几乎保持不变。](https://cdn.sanity.io/images/vr8gru94/production/ef1a2edd25adb202c0a98a1f33a0e72d1295b554-1720x1080.png)

ef 仅搜索一个查询时的构造和搜索时间。当使用较低的 M 值时，对于不同的 efConstruction 值，搜索时间几乎保持不变。

一切看起来都很棒，但是 HNSW 索引的内存使用情况又如何呢？在这里，事情可能会变得不太*有*吸引力。

![使用我们的 Sift1M 数据集，随着 M 值的增加，内存使用情况。 efSearch 和 efConstruction 对内存使用没有影响。](https://cdn.sanity.io/images/vr8gru94/production/e04d23ccd76d8bdc568542bebe75a75e7d36a21e-1480x1050.png)

使用我们的 Sift1M 数据集，随着 M 值的增加，内存使用情况。efSearch 和 efConstruction 对内存使用没有影响。

**efConstruction**和**efSearch**都不影响索引内存使用，只留下**M**。即使**M**的值较低为**2**，我们的索引大小也已超过 0.5GB，**M**为**512**时达到近 5GB，。

因此，尽管 HNSW 产生了令人难以置信的性能，但我们需要权衡高内存使用率以及由此产生的不可避免的高基础设施成本。

# 提高内存使用率和搜索速度

就内存利用率而言，HNSW 并不是最好的索引。但是，如果这很重要并且无法选择使用[其他索引，那么我们可以通过使用](https://www.pinecone.io/learn/series/faiss/vector-indexes/)[乘积量化 (PQ)](https://www.pinecone.io/learn/series/faiss/product-quantization/)压缩向量来改进它。使用 PQ 会减少召回率并增加搜索时间——但与往常一样，人工神经网络的大部分内容都是平衡这三个因素的。

相反，如果我们想提高搜索速度 - 我们也可以做到！我们所做的就是将 IVF 组件添加到我们的索引中。在将 IVF 或 PQ 添加到我们的索引时，有很多需要讨论的内容，因此我们写了一篇[关于索引混合和匹配的整篇文章](https://www.pinecone.io/learn/series/faiss/composite-indexes/)。

这就是本文介绍用于矢量相似性搜索的分层可导航小世界图的内容！现在您已经了解了 HNSW 背后的直觉以及如何在 Faiss 中实现它，您可以继续在自己的矢量搜索应用程序中测试 HNSW 索引，或者使用已准备好矢量搜索的托管解决方案，例如 Pinecone或[OpenSearch](https://www.pinecone.io/) -去吧！

如果您想继续了解有关矢量搜索以及如何使用它来增强您自己的应用程序的更多信息，我们有一整套[学习材料，](https://www.pinecone.io/learn/)旨在帮助您快速了解矢量搜索的世界。

## References

[1] E. Bernhardsson, [ANN Benchmarks](https://github.com/erikbern/ann-benchmarks) (2021), GitHub

[2] W. Pugh, [Skip lists: a probabilistic alternative to balanced trees](https://15721.courses.cs.cmu.edu/spring2018/papers/08-oltpindexes1/pugh-skiplists-cacm1990.pdf) (1990), Communications of the ACM, vol. 33, no.6, pp. 668-676

[3] Y. Malkov, D. Yashunin, [Efficient and robust approximate nearest neighbor search using Hierarchical Navigable Small World graphs](https://arxiv.org/abs/1603.09320) (2016), IEEE Transactions on Pattern Analysis and Machine Intelligence

[4] Y. Malkov et al., [Approximate Nearest Neighbor Search Small World Approach](https://www.iiis.org/CDs2011/CD2011IDI/ICTA_2011/PapersPdf/CT175ON.pdf) (2011), International Conference on Information and Communication Technologies & Applications

[5] Y. Malkov et al., [Scalable Distributed Algorithm for Approximate Nearest Neighbor Search Problem in High Dimensional General Metric Spaces](https://www.researchgate.net/publication/262334462_Scalable_Distributed_Algorithm_for_Approximate_Nearest_Neighbor_Search_Problem_in_High_Dimensional_General_Metric_Spaces) (2012), Similarity Search and Applications, pp. 132-147

[6] Y. Malkov et al., [Approximate nearest neighbor algorithm based on navigable small world graphs](https://publications.hse.ru/mirror/pubs/share/folder/x5p6h7thif/direct/128296059) (2014), Information Systems, vol. 45, pp. 61-68

[7] M. Boguna et al., [Navigability of complex networks](https://arxiv.org/abs/0709.0303) (2009), Nature Physics, vol. 5, no. 1, pp. 74-80

[8] Y. Malkov, A. Ponomarenko, [Growing homophilic networks are natural navigable small worlds](https://arxiv.org/abs/1507.06529) (2015), PloS one

Facebook Research, [Faiss HNSW Implementation](https://github.com/facebookresearch/faiss/blob/main/faiss/impl/HNSW.cpp), GitHub

